In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%pip install nltk
%pip install contractions
%pip install pandas
%pip install tqdm

In [ ]:
# from spellchecker import SpellChecker
import contractions
import urllib.parse
import re
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


class CleanText:

    def __init__(self):
        # self.eng_checker = SpellChecker(language='en')
        self.lemmatizer = nltk.WordNetLemmatizer()
        nltk_stopwords = stopwords.words('english')
        negative_words = ['not', 'no', 'never', 'nor',
                          # location words
                          "off", "out", "over", "under", "up", "down",]
        self.stop_words = [
            word for word in nltk_stopwords if word not in negative_words]

    def _lower_case(self, text):
        return text.lower()

    def _check_url(self, url):
        # check if the url is valid or not
        parsed = urllib.parse.urlparse(url)
        return bool(parsed.scheme and parsed.netloc)

    def _remove_html(self, text):
        # remove all the html tags
        result = text.replace('<.*?>', '')
        return result

    def _expaned_contractions(self, text):
        return contractions.fix(text)

    def _remove_emails(self, text):
        return text.replace(r"[\w\.-]+@[\w\.-]+\.\w+", "")

    def _change_user_name(self, text):
        pattern = re.compile(r"@\w+")
        return re.sub(pattern, "user", text)

    def _replace_urls(self, text):
        # Replace URLs with 'url'
        return re.sub(r'https?://(www\.)?(\w+)(\.\w+)(/\w*)?', 'url', text)

    def _replace_symbol_to_space(self, text):
        # replace all the symbols to space
        return re.sub(r'[^\w\s]', ' ', text)

    def _remove_single_char(self, text):
        # remove all the single characters
        return re.sub(r'\b(?<![a-hj-z])[a-z](?![a-z])\b', ' ', text)

    def _remove_multiple_space(self, text):
        # remove all the multiple spaces
        return re.sub(r'\s+', ' ', text)

    def _remove_number(self, text):
        return re.sub(r'[0-9]', '', text)

    def _remove_repeat_word(self, text):
        # remove all the characters that appear more than 2 times in a word
        return re.sub(r'(.)\1+', r'\1\1', text)

    def _fit_word(self, text):
        # Replace misplassed words
        words = text.split()
        words = [self._correct_word(word) for word in words]
        return ' '.join(words)

    def _correct_word(self, text):
        fixed = self.eng_checker.correction(text)
        if fixed != None:
            return fixed
        return text

    def _remove_emoji(self, text):
        emoji_pattern = re.compile("["
                                   u"\U0001F600-\U0001F64F"  # emoticons
                                   u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                   u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                   u"\U0001F1E0-\U0001F1FF"  # flags
                                   u"\U00002702-\U000027B0"
                                   u"\U000024C2-\U0001F251"
                                   "]+", flags=re.UNICODE)
        return emoji_pattern.sub(r'', text)

    def _remove_stopwords(self, text):
        # Remove stopwords
        word_tokens = nltk.word_tokenize(text)
        filtered_text = [w for w in word_tokens if w not in self.stop_words]
        return ' '.join(filtered_text)

    def _lemmatize_text(self, text):
        # Lemmatize the text
        output=""
        text=text.split(" ")
        for word in text:
          word1 = self.lemmatizer.lemmatize(word, pos = "n")
          word2 = self.lemmatizer.lemmatize(word1, pos = "v")
          word3 = self.lemmatizer.lemmatize(word2, pos = "a")
          word4 = self.lemmatizer.lemmatize(word3, pos = "r")
          output=output + " " + word4
        return str(output.strip())

    def _fit_in_alphabet(self, text):
        regex = re.compile('[^a-zA-Z]')
        # First parameter is the replacement, second parameter is your input string
        return regex.sub(' ', text)

    def clean_lstm(self, text):

        # lọc nhiễu
        text = self._lower_case(text)
        text = self._remove_html(text)
        text = self._replace_urls(text)
        text = self._remove_emails(text)
        text = self._change_user_name(text)
        text = self._remove_emoji(text)
        text = self._remove_repeat_word(text)

        # chuẩn hóa

        text = self._expaned_contractions(text)
        # text = self._fit_word(text)
        text = self._remove_stopwords(text)
        text = self._fit_in_alphabet(text)
        text = self._lemmatize_text(text)

        # loại bỏ phần không cần thiết
        text = self._remove_single_char(text)
        text = self._remove_multiple_space(text)

        return text.strip()

    def clean_w2v(self, text):
        # lọc nhiễu
        text = self._lower_case(text)
        text = self._remove_html(text)
        text = self._replace_urls(text)
        text = self._remove_emails(text)
        text = self._change_user_name(text)
        text = self._remove_emoji(text)
        text = self._remove_repeat_word(text)

        # chuẩn hóa
        text = self._expaned_contractions(text)
        text = self._fit_in_alphabet(text)
        text = self._lemmatize_text(text)

        # loại bỏ phần không cần thiết
        text = self._remove_single_char(text)
        text = self._remove_multiple_space(text)

        return text.strip()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Load Data


In [ ]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

In [ ]:
chunksize = 10**5

chunks = pd.read_csv('/content/drive/MyDrive/KLTN/data/reddit_comments.csv',chunksize=chunksize)


In [ ]:
cleaner = CleanText()
for index, chunk in enumerate(chunks):
  print(f'{index}')
  chunk['body'] = chunk['body'].fillna('no comment').progress_apply(cleaner.clean_w2v)
  chunk.to_csv(f'/content/drive/MyDrive/KLTN/data/reddit_comments_cleaned_{index}.csv');

0


100%|██████████| 100000/100000 [00:35<00:00, 2846.11it/s]


1


100%|██████████| 100000/100000 [00:32<00:00, 3032.57it/s]


2


...

100%|██████████| 100000/100000 [00:39<00:00, 2505.78it/s]


544


100%|██████████| 100000/100000 [00:37<00:00, 2637.46it/s]


545


100%|██████████| 4410/4410 [00:01<00:00, 2713.37it/s]


number 546 files

Make copus file

In [ ]:
path='/content/drive/MyDrive/KLTN/data/preprogress'

In [ ]:
with open(f'{path}/cooking.train','w') as f:
  # reddit comment
  for index in tqdm(range(545)):
    df = pd.read_csv(f'/content/drive/MyDrive/KLTN/data/reddit/reddit_comments_cleaned_{index}.csv').fillna('no comment')
    df.drop_duplicates(subset='body')
    for row in df['body']:
      f.write(row+'\n')

  fb = pd.read_csv(f'/content/drive/MyDrive/KLTN/data/preprogress/fb_comment_cleaned(new).csv').fillna('no comment')
  fb.drop_duplicates(subset='cleaned_text')
  fb.dropna()
  for row in fb['cleaned_text']:
     f.write(row+'\n')




100%|██████████| 545/545 [05:02<00:00,  1.80it/s]


train fasttext

install

In [2]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199672 sha256=5ece642cee8b819265c45b3d694e13740d0b396d77bd07228cd3c2f39504ca36
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [ ]:

import fasttext

training_data_path = '/content/drive/MyDrive/KLTN/data/preprogress/cooking.train'

model = fasttext.train_unsupervised(
    input=training_data_path,
    lr=0.1,             # Learning rate
    dim=100,            # Dimension of word vectors
    epoch=10,           # Number of training epochs
    word_ngrams=3,      # Max length of word ngram
    bucket=200000,      # Number of buckets to use for hashing ngrams
)

model.save_model('/content/drive/MyDrive/KLTN/data/trained_model.bin')